In [1]:
import pandas as pd
from auxiliary.filter_functions import filter_chandra, filter_archival, filter_gaia, filter_ned, filter_simbad, filter_erosita, filter_vizier, filter_Xray_binaries
from auxiliary.search_algorithm import get_wcs_event, off_axis, get_chandra_eef, get_counts_from_event
from results import get_candidate_numbers, get_criteria_table
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad, SimbadClass
import os
import glob
from astropy.table import Table
import numpy as np
from astropy.io import fits
import time
import matplotlib.pyplot as plt
from astropy.stats import poisson_conf_interval
import scipy
# 2372400

In [8]:
date_range = [
    '2015-01-01',
    '2022-04-01'
]

criteria_table = get_criteria_table(from_date=date_range[0], to_date=date_range[1])
candidate_numbers = get_candidate_numbers(from_date=date_range[0], to_date=date_range[1])

total_candidates = candidate_numbers.at['Detections', 'In']

match_rate = [matched/total_candidates for matched in criteria_table['Matched']]
criteria_table_extended = criteria_table.copy()
criteria_table_extended.insert(1, 'Match Rate', match_rate, True)

match_rates_paper = [
    53 / 151,
    83 / 151,
    75 / 151,
]

print(date_range, candidate_numbers, sep='\n', end='\n\n')

print(
    'Match Rates (Paper II):',
    f'Archival: {match_rates_paper[0]:.2f}',
    f'Gaia: {match_rates_paper[1]:.2f}',
    f'NEDSIMVIZ: {match_rates_paper[2]:.2f}',
    sep='\n'
)

# !!! As you can see the match rates for Gaia and NEDSIMVIZ are relatively close to the paper. 
# But I get way les matches for archival data.
# Maybe a higher search radius?
# Because right now there are in total still 429 candidates, which is way too many to check by hand.

criteria_table_extended

['2015-01-01', '2022-04-01']
                    Total Before    In After
Observations         9799   4992  3311  1496
Detections           2802   2235   466   101
Candidates no match   429    355    57    17

Match Rates (Paper II):
Archival: 0.35
Gaia: 0.55
NEDSIMVIZ: 0.50


,Matched,Match Rate,Unique Matched,Removed,Remaining
Archival X-ray date,305,0.654506,79,0,161
Cross-match with stars/Gaia,235,0.504292,38,82,79
NED + SIMBAD + VizieR,218,0.467811,12,0,57


In [3]:
criteria_seperate = [
    ('archival', ['archival_match']),
    ('chandra', ['chandra_match']),
    ('erosita', ['erosita_match']),
    ('gaia', ['gaia_match']),
    ('ned', ['ned_match']),
    ('simbad', ['simbad_match']),
    ('vizier', ['vizier_match']),
    # ('Xray_binaries', 'Xray_binaries_match'),
]

criteria_table_seperate = get_criteria_table(from_date='2015-01-01', to_date='2022-04-01', criteria=criteria_seperate)
criteria_table_seperate

,Matched,Unique Matched,Removed,Remaining
archival,118,6,118,347
chandra,285,73,186,161
erosita,1,0,0,161
gaia,235,38,82,79
ned,110,0,10,69
simbad,18,1,1,66
vizier,196,11,9,57


In [4]:
detections = pd.read_csv('output/detections_w20.txt',
                         header=0, sep=' ', dtype=str)

detections = detections[detections['ObsId'] == '8490'][0:1]
for index, detection in detections.iterrows():
    print(index, filter_Xray_binaries(detection, True))

  ID           Name          RAJ2000   DEJ2000    Flux    b_Flux   B_Flux  Flagp Flagu Flagn    c       LX      b_LX     B_LX  
                               deg       deg    mW / m2  mW / m2  mW / m2                            1e-07 W  1e-07 W  1e-07 W 
----- --------------------- --------- --------- -------- -------- -------- ----- ----- ----- ------- -------- -------- --------
46957 2CXO J132458.3-430226 201.24330 -43.04060        0        0 9.78e-17     0     1     0 0.50004        0        0 1.45e+35
0 True
